In [1]:
# Imports
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt

In [2]:
#make sure we are running off of the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
# Settings to use for both model
beta_1 = 0.0
betas = [0.8, 0.9, 0.95, 0.99]
beta_2 = 0.8
batchsize = 8
batchSizes = [8, 16, 32]
epochs = 100

In [ ]:
# Main code
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def show_some_pictures(j ):
    # get some random training images
    dataiter = iter(trainloader)
    images, labels = dataiter.next()

    # show images
    imshow(torchvision.utils.make_grid(images))
    # print labels
    print(' '.join('%5s' % classes[labels[j]] for j in range(j)))


def random_test():
    dataiter = iter(testloader)
    images, labels = dataiter.next()

    # print images
    imshow(torchvision.utils.make_grid(images))
    print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

    net = Net()
    net.load_state_dict(torch.load(PATH))
    outputs = net(images)

    _, predicted = torch.max(outputs, 1)

    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                                  for j in range(4)))

def train(net, noe, filepath_trainloss, filepath_testacc, filepath_trainacc, initial_lr,  batchsize, device):
    doc = open(filepath_trainloss+'.txt', "w")
    doc2 = open(filepath_testacc+'.txt', "w")
    doc3 = open(filepath_trainacc+'.txt', "w")
    check_interval=1000
    batch_number = int(6000*8/(batchsize*check_interval))
    # print(batch_number)
    training_loss_vec = [] #np.zeros(noe*check_interval)
    train_acc_vec = [] #np.zeros(noe*check_interval)
    test_acc_vec = [] #np.zeros(noe*check_interval)
    for epoch in range(noe):  # loop over the dataset multiple times
        time_begin  = time.time()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % check_interval == (check_interval-1):    # print every 2000 mini-batches
                time_end = time.time()
                time_elapsed = time_end - time_begin
                time_begin = time.time()
                print('[%d, %5d] loss: %.3f, Time Taken %.3f' %
                      (epoch + 1, i + 1, running_loss / check_interval, time_elapsed))
                running_loss = 0.0
        if epoch % 10 == 0:
            time_end = time.time()
            time_elapsed = time_end - time_begin
            time_begin = time.time()
            print("\tTesting Time: ", time_elapsed)
            training_loss_vec.append(running_loss/check_interval)
            train_acc = train_accuracy(net)
            train_acc_vec.append(train_acc)
            test_acc = test_accuracy(net)
            test_acc_vec.append(test_acc)
            print(running_loss / check_interval, file=doc)
            print(test_acc, file=doc2)
            print(train_acc, file=doc3)
            net.train()
        if epoch % 1 == 0:
            for p in optimizer.param_groups:
                p['lr'] = initial_lr/np.sqrt(1+epoch)
    training_loss_vec.append(running_loss/check_interval)
    train_acc = train_accuracy(net)
    train_acc_vec.append(train_acc)
    test_acc = test_accuracy(net)
    test_acc_vec.append(test_acc)
    print(running_loss / check_interval, file=doc)
    print(test_acc, file=doc2)
    print(train_acc, file=doc3)
    doc.close()
    doc2.close()

    xvar = np.arange(len(training_loss_vec)) * 10
    plt.figure(1)
    training_title = "Training Loss Batch Size:" + str(batchsize) + " Beta2:" + str(beta_2)
    plt.title(training_title, fontsize=15)
    plt.xlabel('epochs')
    plt.ylabel('training loss')
    plt.plot(xvar, np.array(training_loss_vec))
    plt.savefig(filepath_trainloss+'.png')
    plt.close() 

    plt.figure(2)
    testing_title = "Test Accuracy Batch Size:" + str(batchsize) + " Beta2:" + str(beta_2)
    plt.title(testing_title, fontsize=15)
    plt.xlabel('epochs')
    plt.ylabel('test accuracy(%)')
    plt.plot(xvar, np.array(test_acc_vec), label="Test accuracy")
    plt.legend()
    plt.savefig(filepath_testacc + '.png')

    plt.figure(2)
    both_title = "Test Accuracy Batch Size:" + str(batchsize) + " Beta2:" + str(beta_2)
    plt.title(both_title, fontsize=15)
    plt.xlabel('epochs')
    plt.ylabel('training accuracy(%)')
    plt.plot(xvar, np.array(train_acc_vec), label="Training accuracy")
    plt.legend()
    plt.savefig(filepath_trainacc + '.png')
    plt.close() 

def train_accuracy(net):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in trainloader:
            images, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def test_accuracy(net):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total



In [ ]:
# data sets
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
# resnet
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
net = net.to(device)
net.train()
# cifar_resnet.py
criterion = nn.CrossEntropyLoss()
PATH = './cifar_net_Adam.pth'
for beta in betas:
    beta_2 = beta
    for batchSize in batchSizes:
        batchsize = batchSize
        net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
        net = net.to(device)
        optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(beta_1, beta), eps=1e-08, weight_decay=0, amsgrad=False)

        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batchsize,
                                          shuffle=True, num_workers=8,  pin_memory=True)

        testloader = torch.utils.data.DataLoader(testset, batch_size=batchsize,
                                         shuffle=False, num_workers=8,  pin_memory=True)
        train(net, epochs, "Exp1-training-loss-beta1="+str(beta_1)+";beta2="+str(beta)+";bs="+str(batchSize), "Exp1-test-accuracy-beta1="+str(beta_1)+";beta2="+str(beta_2)+";bs="+str(batchsize), "Exp1-train-accuracy-beta1="+str(beta_1)+";beta2="+str(beta_2)+";bs="+str(batchsize), 0.001, batchsize, device)
        torch.save(net.state_dict(), PATH)
print('Finished Training')

Files already downloaded and verified
Files already downloaded and verified


Using cache found in C:\Users\Aria/.cache\torch\hub\pytorch_vision_v0.6.0
Using cache found in C:\Users\Aria/.cache\torch\hub\pytorch_vision_v0.6.0


[1,  1000] loss: 2.205, Time Taken 42.065
[1,  2000] loss: 1.947, Time Taken 8.568
[1,  3000] loss: 1.791, Time Taken 8.478
[1,  4000] loss: 1.722, Time Taken 8.438
[1,  5000] loss: 1.632, Time Taken 8.398
[1,  6000] loss: 1.597, Time Taken 8.432
	Testing Time:  2.6630663871765137
[2,  1000] loss: 1.531, Time Taken 34.437
[2,  2000] loss: 1.534, Time Taken 8.434
[2,  3000] loss: 1.498, Time Taken 8.694
[2,  4000] loss: 1.469, Time Taken 9.063
[2,  5000] loss: 1.449, Time Taken 9.063
[2,  6000] loss: 1.487, Time Taken 8.828
[3,  1000] loss: 1.329, Time Taken 35.201
[3,  2000] loss: 1.291, Time Taken 8.433
[3,  3000] loss: 1.306, Time Taken 8.682
[3,  4000] loss: 1.279, Time Taken 8.511
[3,  5000] loss: 1.266, Time Taken 8.499
[3,  6000] loss: 1.244, Time Taken 8.457
[4,  1000] loss: 1.177, Time Taken 33.858
[4,  2000] loss: 1.184, Time Taken 8.400
[4,  3000] loss: 1.174, Time Taken 8.405
[4,  4000] loss: 1.179, Time Taken 8.412
[4,  5000] loss: 1.137, Time Taken 8.403
[4,  6000] loss: 1

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001C6BCC7C0E0>
Traceback (most recent call last):
  File "c:\Users\Aria\Desktop\CS536FinalProject\CS536_FinalProject\LocalRun\gpu_env\Lib\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "c:\Users\Aria\Desktop\CS536FinalProject\CS536_FinalProject\LocalRun\gpu_env\Lib\site-packages\torch\utils\data\dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


[8,  1000] loss: 0.936, Time Taken 34.190
[8,  2000] loss: 0.946, Time Taken 8.569
[8,  3000] loss: 0.955, Time Taken 8.480
[8,  4000] loss: 0.931, Time Taken 8.467
[8,  5000] loss: 0.937, Time Taken 8.453
[8,  6000] loss: 0.965, Time Taken 8.460
[9,  1000] loss: 0.890, Time Taken 33.722
[9,  2000] loss: 0.923, Time Taken 8.416
[9,  3000] loss: 0.934, Time Taken 8.502
[9,  4000] loss: 0.904, Time Taken 8.474
[9,  5000] loss: 0.918, Time Taken 8.445
[9,  6000] loss: 0.960, Time Taken 8.463
[10,  1000] loss: 0.874, Time Taken 33.885
[10,  2000] loss: 0.911, Time Taken 8.420
[10,  3000] loss: 0.898, Time Taken 8.419
[10,  4000] loss: 0.885, Time Taken 8.431
[10,  5000] loss: 0.866, Time Taken 8.449
[10,  6000] loss: 0.869, Time Taken 8.380
[11,  1000] loss: 0.791, Time Taken 33.748
[11,  2000] loss: 0.839, Time Taken 8.500
[11,  3000] loss: 0.817, Time Taken 8.501
[11,  4000] loss: 0.871, Time Taken 8.489
[11,  5000] loss: 0.826, Time Taken 8.510
[11,  6000] loss: 0.825, Time Taken 8.493


...

[1,  1000] loss: 2.189, Time Taken 33.528
[1,  2000] loss: 1.914, Time Taken 8.632
[1,  3000] loss: 1.776, Time Taken 8.483
[1,  4000] loss: 1.651, Time Taken 8.734
[1,  5000] loss: 1.582, Time Taken 8.698
[1,  6000] loss: 1.493, Time Taken 8.622
	Testing Time:  2.752192974090576
[2,  1000] loss: 1.392, Time Taken 33.776
[2,  2000] loss: 1.355, Time Taken 8.737
[2,  3000] loss: 1.330, Time Taken 8.625
[2,  4000] loss: 1.282, Time Taken 8.605
[2,  5000] loss: 1.257, Time Taken 8.609
[2,  6000] loss: 1.212, Time Taken 8.574
[3,  1000] loss: 1.068, Time Taken 34.049
[3,  2000] loss: 1.045, Time Taken 8.596
[3,  3000] loss: 1.048, Time Taken 8.634
[3,  4000] loss: 1.029, Time Taken 8.520
[3,  5000] loss: 1.008, Time Taken 8.649
[3,  6000] loss: 0.983, Time Taken 8.624
[4,  1000] loss: 0.906, Time Taken 34.067
[4,  2000] loss: 0.860, Time Taken 8.696
[4,  3000] loss: 0.864, Time Taken 8.580
[4,  4000] loss: 0.881, Time Taken 8.578
[4,  5000] loss: 0.876, Time Taken 8.617
[4,  6000] loss: 0.

Using cache found in C:\Users\Aria/.cache\torch\hub\pytorch_vision_v0.6.0


[1,  1000] loss: 1.874, Time Taken 34.078
[1,  2000] loss: 1.510, Time Taken 8.745
[1,  3000] loss: 1.321, Time Taken 8.783
	Testing Time:  1.6971840858459473
[2,  1000] loss: 1.154, Time Taken 34.091
[2,  2000] loss: 1.074, Time Taken 8.633
[2,  3000] loss: 1.018, Time Taken 8.728
[3,  1000] loss: 0.852, Time Taken 34.290
[3,  2000] loss: 0.827, Time Taken 8.875
[3,  3000] loss: 0.797, Time Taken 8.792
[4,  1000] loss: 0.680, Time Taken 34.127
[4,  2000] loss: 0.674, Time Taken 8.754
[4,  3000] loss: 0.659, Time Taken 8.594
[5,  1000] loss: 0.546, Time Taken 34.052
[5,  2000] loss: 0.555, Time Taken 8.724
[5,  3000] loss: 0.538, Time Taken 8.841
[6,  1000] loss: 0.434, Time Taken 34.828
[6,  2000] loss: 0.444, Time Taken 8.723
[6,  3000] loss: 0.460, Time Taken 8.726
[7,  1000] loss: 0.343, Time Taken 34.321
[7,  2000] loss: 0.357, Time Taken 8.723
[7,  3000] loss: 0.363, Time Taken 8.781
[8,  1000] loss: 0.266, Time Taken 34.181
[8,  2000] loss: 0.280, Time Taken 8.792
[8,  3000] los

Using cache found in C:\Users\Aria/.cache\torch\hub\pytorch_vision_v0.6.0


[1,  1000] loss: 1.614, Time Taken 33.866
	Testing Time:  5.557582855224609
[2,  1000] loss: 1.037, Time Taken 34.229
[3,  1000] loss: 0.761, Time Taken 34.322
[4,  1000] loss: 0.580, Time Taken 34.469
[5,  1000] loss: 0.456, Time Taken 34.047
[6,  1000] loss: 0.337, Time Taken 34.160
[7,  1000] loss: 0.251, Time Taken 34.099
[8,  1000] loss: 0.179, Time Taken 34.355
[9,  1000] loss: 0.137, Time Taken 34.053
[10,  1000] loss: 0.100, Time Taken 34.261
[11,  1000] loss: 0.081, Time Taken 34.064
	Testing Time:  5.557913780212402
[12,  1000] loss: 0.068, Time Taken 34.738
[13,  1000] loss: 0.055, Time Taken 34.200
[14,  1000] loss: 0.049, Time Taken 34.373
[15,  1000] loss: 0.044, Time Taken 34.159
[16,  1000] loss: 0.035, Time Taken 34.300
[17,  1000] loss: 0.032, Time Taken 34.216
[18,  1000] loss: 0.032, Time Taken 34.324
[19,  1000] loss: 0.030, Time Taken 34.260
[20,  1000] loss: 0.026, Time Taken 34.354
[21,  1000] loss: 0.023, Time Taken 34.048
	Testing Time:  5.791679620742798
[22,

In [ ]:
# cifar_resnet_SGD.py
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=beta_1, dampening=0, weight_decay=0, nesterov=False)
PATH = './cifar_net_SGD.pth'
for batchSize in batchSizes:
    train(net, epochs, "Exp1-training-loss-beta1="+str(beta_1)+";bs="+str(batchSize), "Exp1-test-accuracy-beta1="+str(beta_1)+";bs="+str(batchSize), "Exp1-train-accuracy-beta1="+str(beta_1)+";bs="+str(batchSize), 0.001, batchSize, device)
    torch.save(net.state_dict(), PATH)
print('Finished Training')